In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from matplotlib import pyplot as plt
import numpy as np
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import imageio

In [0]:
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5,),(0.5,))
                ])
to_image = transforms.ToPILImage()
trainset = MNIST(root='./data/', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

device = 'cuda'

In [0]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.n_features = 100
        self.n_out = 784
        self.model = nn.Sequential(
                    nn.Linear(self.n_features, 256),
                    nn.LeakyReLU(0.2),
                    nn.Linear(256, 512),
                    nn.LeakyReLU(0.2),
                    nn.Linear(512, 1024),
                    nn.LeakyReLU(0.2),
                    nn.Linear(1024, self.n_out),
                    nn.Tanh()
                    )
    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 1, 28, 28)
        return x

class Critic(nn.Module):
    def __init__(self):
        super(Critic, self).__init__()
        self.n_in = 784
        self.n_out = 1
        self.model = nn.Sequential(
                    nn.Linear(self.n_in, 1024),
                    nn.LeakyReLU(0.2),
                    nn.Dropout(0.3),
                    nn.Linear(1024, 512),
                    nn.LeakyReLU(0.2),
                    nn.Dropout(0.3),
                    nn.Linear(512, 256),
                    nn.LeakyReLU(0.2),
                    nn.Dropout(0.3),
                    nn.Linear(256, self.n_out)
                    )
    def forward(self, x):
        x = x.view(-1, 784)
        x = self.model(x)
        return x

In [0]:
generator = Generator()
critic = Critic()

generator.to(device)
critic.to(device)

alpha = 5e-5
g_optim = optim.RMSprop(generator.parameters(), lr=alpha)
c_optim = optim.RMSprop(critic.parameters(), lr=alpha)

g_losses = []
c_losses = []
images = []

def noise(n, n_features=100):
    return torch.randn(n, n_features).to(device)

In [0]:
def train_critic(optimizer, real_data, fake_data, c=0.01):
    optimizer.zero_grad()
    error_real = critic(real_data).mean()
    error_fake = critic(fake_data).mean()
    total_error = -(error_real - error_fake)
    total_error.backward()
    optimizer.step()
    for p in critic.parameters():
        p.data.clamp_(-c, c)
    return -total_error

def train_generator(optimizer, fake_data):
    optimizer.zero_grad()
    error = -critic(fake_data).mean()
    error.backward()
    optimizer.step()
    return error

In [0]:
num_epochs = 250
n_critic = 5
test_noise = noise(64)

generator.train()
critic.train()
for epoch in range(num_epochs):
    g_error = 0.0
    c_error = 0.0
    for i, data in enumerate(trainloader):
        imgs, _ = data
        n = len(imgs)
        fake_data = generator(noise(n)).detach()
        real_data = imgs.to(device)
        c_error += train_critic(c_optim, real_data, fake_data)
        
        if (i+1)%n_critic==0:
            fake_data = generator(noise(n))
            g_error += train_generator(g_optim, fake_data)
    if epoch%5==0:
        img = generator(test_noise).cpu().detach()
        img = make_grid(img)
        images.append(to_image(img))
        g_losses.append(g_error)
        c_losses.append(c_error)
        plt.clf()
        plt.plot(g_losses, label='Generator Losses')
        plt.plot(c_losses, label='Critic Losses')
        plt.legend()
        plt.savefig('loss.png')
        imageio.mimsave('progress.gif', [np.array(i) for i in images])
    print('Epoch {}: G_loss: {:.4f} C_loss: {:.4f}'.format(epoch, g_error, c_error))
    
print('Training Finished')
torch.save(generator.state_dict(), 'mnist_generator.pth')